In [6]:
import os

"""
CKY Parsing 테이블의 각 칸에 저장될 노드. Shared Packed Tree를 따른다. 
children_list: 노드의 가능한 자식 Tuple의 Array. 즉 [(A,B),(C,D)...]
"""
class CKYPackedNode:
    """
    A => B C
    symbol: A에 해당. 단어의 품사 심볼
    child1: B에 해당. 이 값은 CKYPackedNode가 될 수 있음.
    child2: C에 해당. 이 값은 CKYPackedNode가 될 수 있음.
    """
    def __init__(self, symbol, child1=None, child2=None):
        self.symbol = symbol
        if child1 is not None:
            self.children_list = [[child1, child2]]
        else:
            self.children_list = []
            
        # 노드의 고유 Id값을 static하게 counting하여 할당
        self.id = CKYPackedNode._id
        CKYPackedNode._id = CKYPackedNode._id + 1

    # 노드의 문자열 표현. "1. (symbol, [(1,2),(3,4)])" 형태로, children_list는 노드의 ID를 참조하여 Tuple Array로 표현된다. 
    def __str__(self):
        children_set_str = []
        for children in self.children_list:
            valid_children = list(filter(lambda x: x is not None, children))
            valid_children_str = ','.join(list(map(lambda node: node if type(node) == str else '' if node is None else str(node.id), valid_children)))
            children_set_str.append('(' + valid_children_str + ')')
        return str(self.id) + '. (' + self.symbol + ',' + '[' + ','.join(children_set_str) + '])'
    
    # 해당 노드로 부터 파생될 수 있는 모든 Parse Tree에 대해 각각 전체 트리구조 문자열을 표현하여 배열로 리턴. 
    def tree_str(self):
        tree_str_list = []        
        for children in self.children_list:
            left_str_list = []
            right_str_list = []
            if type(children[0]) == str:
                left_str_list.append(children[0])
            else:
                left_str_list.extend(children[0].tree_str())
                    
            if children[1] is None:
                right_str_list.append('')
            else:
                if type(children[1]) == str:
                    right_str_list.append(children[1])
                else:
                    right_str_list.extend(children[1].tree_str())
            
            for left_str in left_str_list:
                for right_str in right_str_list:
                    if right_str != '':
                        tree_str_list.append(f'({self.symbol}, {left_str}, {right_str})')
                    else:
                        tree_str_list.append(f'({self.symbol}, {left_str})')
        return tree_str_list

    
# 문법 사전
class GrammarDict:
    def __init__(self):
        self.grammar_map = {}
        
    """
    symbol => child1 child2 에 해당하는 문법 추가
    """
    def put_grammar(self, symbol, child1, child2=None):
        if symbol not in self.grammar_map:
            self.grammar_map[symbol] = []    
        grammar_list = self.grammar_map[symbol]
        grammar_list.append([child1, child2])
    
    """
    symbol에서 파생 가능한 children 리스트 구하기
    """
    def get_children(self, symbol):
        return self.grammar_map.get(symbol, None)
    
    """
    child1 child2를 표현할 수 있는 단일 심볼 목록을 재귀적으로 모두 구하기
    """
    def get_symbols(self, child1, child2=None):
        symbols = set([])
        
        for symbol, children_set in self.grammar_map.items():
            for children in children_set:
                if children[0] == child1 and children[1] == child2:
                    symbols.add(symbol)
        
        if len(symbols) > 0:
            for symbol in symbols:
                symbols = symbols.union(self.get_symbols(symbol))
        return symbols
    
    """
    symbol => child1 child2에 해당하는 문법 유효 검사
    """
    def is_matched(self, symbol, child1, child2=None):
        if symbol not in self.grammar_map:
            return False
        for grammar in self.grammar_map[symbol]:
            if grammar[0] == symbol and grammar[1] == child1 and grammar[2] == child2:
                return True
        return False
        

"""
CKY Parse를 수행하는 클래스
"""
class CKYParser:
    """
    grammar_dict: Parsing할때 필요한 사전 (grammarDict)
    """
    def __init__(self, grammar_dict):
        self.grammar_dict = grammar_dict
        
    """
    CKY Parse 후 전체 문장에 대한 CKYPackedNode 목록 리턴 (S symbol만)
    used_grammar_output: 분석에 사용된 문법 인덱스 로깅파일이름
    print_table: CKY Table 콘솔 출력 여부    
    """
    def parse(self, sentence, used_grammar_output=None, print_table=False):
        sentence = sentence.strip()
        o = None
        if used_grammar_output is not None:
            o = open(used_grammar_output, "a+")
        print(f'####### {sentence} #######')
        if o is not None and o.closed == 0:
            o.writelines([f'####### {sentence} #######', '\n'])
        CKYPackedNode._id = 1
        words = list(map(lambda word: word.strip(), sentence.split(" ")))
        cky_table = [[[] for j in range(0, i)] for i in range(len(words), 0, -1)]
        for c in range(0, len(words)):
            for r in range(c, -1, -1):
                is_last_cell = (c == len(words) - 1) and r == 0
                if c == r:
                    symbols = self.grammar_dict.get_symbols(words[c])
                    if len(symbols) > 0:
                        for symbol in symbols:
                            if symbol == 'S':
                                continue
                            node = CKYPackedNode(symbol, words[c])
                            cky_table[r][c - r].append(node)
                            print(str(node))
                            if o is not None and o.closed == 0:
                                o.writelines([str(node) + '\n'])
                            
                    continue
                for i in range(0, c - r):
                    left_nodes = cky_table[r][i]
                    right_nodes = cky_table[r + i + 1][c - r - i - 1]
                    
                    if len(left_nodes) > 0 and len(right_nodes) > 0:
                        for left_node in left_nodes:
                            for right_node in right_nodes:
                                
                                symbols = self.grammar_dict.get_symbols(left_node.symbol, right_node.symbol)
                                if len(symbols) > 0:
                                    for symbol in symbols:
                                        if is_last_cell == False and symbol == 'S':
                                            continue
                                        already_exists = False
                                        matched_node = None
                                        for cky_node in cky_table[r][c - r]:
                                            if cky_node.symbol == symbol:    
                                                already_exists = True
                                                matched_node = cky_node
                                                matched_node.children_list.append([left_node, right_node])
                                                break
                                        if already_exists == False:
                                            cky_table[r][c - r].append(CKYPackedNode(symbol, left_node, right_node))
                                            matched_node = cky_table[r][c - r][-1]
                                            
                                        print(str(matched_node), ' # packed !' if already_exists else '')
                                        if o is not None and o.closed == 0:
                                            o.writelines([str(matched_node), ' # packed !' if already_exists else '', '\n'])
        print()
        if o is not None and o.closed == 0:
            o.writelines(['\n'])
            o.close()
            
        
        if print_table == True:
            o = open('table.txt', 'a+')
            o.writelines([f'####### {sentence} #######', '\n'])
            for word in words:
                print(f'{word:20}\t', end='')
                o.writelines([f'{word:20}\t'])
            print()
            o.writelines(['\n'])
            padding = 0
            for row in cky_table:
                for i in range(0, padding):
                    print(f'{"":20}\t', end='')
                    o.writelines([f'{"":20}\t'])
                for col in row:
                    strs = ','.join(list(map(lambda node: node.symbol, col)))
                    print(f'{strs:20}\t', end='')
                    o.writelines([f'{strs:20}\t'])
                print()
                o.writelines(['\n'])
                padding = padding + 1
            o.close()
        return cky_table[0][len(words) - 1]

# 문법사전 로딩
grammar_dict = GrammarDict()
parser = CKYParser(grammar_dict)
grammar_file = open("grammar.txt", encoding='utf-8')
for line in grammar_file.readlines():
    if line.find('->') == -1:
        continue
    left, right = line.split("->")
    symbol = left.strip()
    right = right.strip()
    children = right.split(" ")
    if len(children) == 2:
        grammar_dict.put_grammar(symbol, children[0], children[1])
    else:
        grammar_dict.put_grammar(symbol, children[0])
grammar_file.close()

# 구구조 분석시 사용한 문법인덱스 로그파일 삭제
if os.path.exists("used_grammar.txt"):
    os.remove("used_grammar.txt")

# CKY Parse Table 로그파일 삭제
if os.path.exists("table.txt"):
    os.remove("table.txt")
    
# 완성된 Parse Tree 저장
output_file = open("output.txt", 'w+', encoding='utf-8')

# 구조 분석할 문장 읽기
input_file = open("input.txt", encoding='utf-8')
for line in input_file.readlines():
    # 주석 무시
    if line.startswith('#'):
        continue
    # 좌우 whitespace 제거를 해야 맨 앞, 뒤단어에 대한 사전 매칭에 문제가 없다.
    line = line.strip()
    # 문장에 대해 CKY Parse 수행 후, 사용된 문법 인덱스 used_grammar.txt에 로깅
    trees = parser.parse(line, "used_grammar.txt", True)
    output_file.writelines([f'####### {line} #######', '\n'])
    for tree in trees:
        # 최종적으로 Parse Tree의 root가 Sentence가 아닌 것은 무시
        if tree.symbol == 'S':
            # 가능한 Parse Tree 각각 파일에 출력
            for tree_str in tree.tree_str():
                print(tree_str);
                output_file.writelines([tree_str, '\n'])
    output_file.writelines(['\n'])
input_file.close()
output_file.close()


####### I saw a man on the hill with the telescope #######
1. (n,[(I)])
2. (NP,[(I)])
3. (n,[(saw)])
4. (VP,[(saw)])
5. (v,[(saw)])
6. (NP,[(saw)])
7. (NP,[(2,4)]) 
8. (DT,[(a)])
9. (det,[(a)])
10. (n,[(man)])
11. (VP,[(man)])
12. (v,[(man)])
13. (NP,[(man)])
14. (NP,[(8,13)]) 
15. (VP,[(4,14)]) 
16. (NP,[(2,15)]) 
17. (P,[(on)])
18. (p,[(on)])
19. (DT,[(the)])
20. (det,[(the)])
21. (n,[(hill)])
22. (NP,[(hill)])
23. (NP,[(19,22)]) 
24. (PP,[(17,23)]) 
25. (VP,[(11,24)]) 
26. (NP,[(13,24)]) 
27. (NP,[(8,26)]) 
27. (NP,[(8,26),(14,24)])  # packed !
28. (VP,[(4,27)]) 
28. (VP,[(4,27),(15,24)])  # packed !
29. (NP,[(2,28)]) 
29. (NP,[(2,28),(16,24)])  # packed !
30. (P,[(with)])
31. (p,[(with)])
32. (DT,[(the)])
33. (det,[(the)])
34. (n,[(telescope)])
35. (NP,[(telescope)])
36. (NP,[(32,35)]) 
37. (PP,[(30,36)]) 
38. (NP,[(22,37)]) 
39. (NP,[(19,38)]) 
39. (NP,[(19,38),(23,37)])  # packed !
40. (PP,[(17,39)]) 
41. (VP,[(11,40)]) 
42. (NP,[(13,40)]) 
41. (VP,[(11,40),(25,37)])  # packed !
